In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt   
# doesn't work on my laptop
from tistools import read_inputfile, get_LMR_interfaces, read_pathensemble, get_weights
from tistools import set_tau_distrib, set_tau_first_hit_M_distrib, cross_dist_distr, pathlength_distr
from tistools import ACCFLAGS, REJFLAGS

from tistools import get_lmr_masks, get_generation_mask, get_flag_mask, select_with_masks
from tistools import unwrap_by_weight, running_avg_local_probs, get_local_probs, get_globall_probs, get_global_probz
from tistools import make_plot_trajs

from pprint import pprint    # to print the vars of the pathensemble object

# TODO: change to istar_analysis when finished
from istar_test import *

%matplotlib qt

In [2]:
import logging

logger = logging.getLogger(__name__)

In [3]:
%autoreload 2   
# something with pip install -e .

# Reading

In [53]:
# zero_minus_one = True if lambda_-1 interface is set
# zero_minus_one = False if lambda_-1 interface is not set

# data the maze
# ---------------
# indir = indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pastime1/11.2024_StapleTIS/simulations/flat/sim_istar0br2108/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istar2cosbr_2108/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarz_g5/"
# indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istar0br2108/"

indir = "/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209/"
# indir = "/run/user/1001/gvfs/smb-share:server=files.ugent.be,share=eliawils,user=eliawils/shares/tw06_biommeda_pastime1/11.2024_StapleTIS/simulations/maze2D/sim_istarmaze_kick2708/"
zero_minus_one = False
inputfile = indir + "logging.log"


import os
import glob
os.chdir(indir)
print(os.getcwd())

folders = glob.glob(indir + "/0[0-9][0-9]")
folders = sorted(folders)
print(folders)

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209
['/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209/000', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209/001', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209/002', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209/003', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209/004', '/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarwell2_0209/005']


In [46]:
# !!! last lines !!!  allow to speed up this notebook
# pe.set_orders(load=False...)  -> 1st time you run the code, this will store npy files
# pe.set_orders(load=True...)  -> next time you run the code, you can read npy files

# Reading all input
#===================
interfaces, zero_left, timestep = read_inputfile(inputfile)
LMR_interfaces, LMR_strings = get_LMR_interfaces(interfaces, zero_left)
pathensembles = []
for i,fol in enumerate(folders):
    print("#"*80)
    print(fol)
    pe = read_pathensemble(fol+"/pathensemble.txt")
    pe.set_name(fol)
    pe.set_interfaces([LMR_interfaces[i], LMR_strings[i]])
    if i==0:
        pe.set_zero_minus_one(zero_minus_one)   # TODO this is never used
        pe.set_in_zero_minus(True)
    if i==1:
        pe.set_in_zero_plus(True)
    w, _ = get_weights(pe.flags, ACCFLAGS, REJFLAGS, verbose = False)
    pe.set_weights(w)
    print("pathensemble info: ")
    pprint(vars(pe))
    pathensembles.append(pe)
    # read order parameters order.txt/order.npy into path ensemble object
    #pe.set_orders(load=False, acc_only=True, save=False) # if saving doesn't work
    #### CHANGE HERE ####
    # pe.set_orders(load=False, acc_only=True, save=True) # for the 1st time
    # pe.set_orders(load=True, acc_only=True) # for the next times, save=True/False is not important

################################################################################
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/PyRETIS3/toytis/simulations/sim_istarz_g5/000
pathensemble info: 
{'cyclenumbers': array([     0,      1,      2, ...,  99998,  99999, 100000]),
 'dirs': array([-1.,  1.,  1., ...,  1., -1., -1.]),
 'flags': array(['ACC', 'BTL', 'BTL', ..., 'ACC', 'FTL', 'FTL'], dtype='<U3'),
 'generation': array(['ld', 'sh', 'sh', ..., 'sh', 'sh', 'sh'], dtype='<U2'),
 'has_zero_minus_one': False,
 'in_zero_minus': True,
 'in_zero_plus': False,
 'interfaces': [[0.05, 0.05, 0.05], ['l_[0]', 'l_[0]', 'l_[0]']],
 'istar_idx': array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]]),
 'lambmaxs': array([0.05026212, 0.02820185, 0.0331673 , ..., 0.05009164, 0.05011043,
       0.05063254]),
 'lambmins': array([-0.00493005, -0.00379134, -0.00368015, ..., -0.00320428,
       -0.00745263, -0.00268888]),
 'lengths': array([ 295,  353,  415, 

In [47]:
for pe in pathensembles:
    print(f"ensemble {pathensembles.index(pe)}")
    print(max(pe.lengths))

for i in range(len(pathensembles)):
    accmask = get_flag_mask(pathensembles[i], "ACC")
    loadmask = get_generation_mask(pathensembles[i], "ld")
    mask = get_lmr_masks(pathensembles[i])
    print(i, np.average(select_with_masks(pathensembles[i].lengths, [~loadmask, accmask])))
    print("LMRlen", i, np.average(select_with_masks(pathensembles[i].lengths, [~loadmask, accmask])))

ensemble 0
7370
ensemble 1
8383
ensemble 2
8383
ensemble 3
7339
ensemble 4
8113
ensemble 5
8501
ensemble 6
9202
ensemble 7
9324
ensemble 8
8596
ensemble 9
9267
ensemble 10
9187
0 789.1265349258539
LMRlen 0 789.1265349258539
1 180.12012991720678
LMRlen 1 180.12012991720678
2 1324.0081090372287
LMRlen 2 1324.0081090372287
3 1628.446435134142
LMRlen 3 1628.446435134142
4 1848.0398501362397
LMRlen 4 1848.0398501362397
5 2067.522290699536
LMRlen 5 2067.522290699536
6 2268.728101911798
LMRlen 6 2268.728101911798
7 2303.906652720491
LMRlen 7 2303.906652720491
8 2248.2358733580577
LMRlen 8 2248.2358733580577
9 2125.05311305545
LMRlen 9 2125.05311305545
10 1847.7480229845821
LMRlen 10 1847.7480229845821


In [48]:
# plocrepptis, plocistar = cprobs_repptis_istar(pathensembles, interfaces, len(interfaces))
plocrepptis, plocistar = cprobs_repptis_istar2(pathensembles, interfaces, len(interfaces))

weights:
accepted      59613
rejected      40388
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 100001
sum weights ensemble 0= 0.0
weights:
accepted      68968
rejected      31033
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 100001
sum weights ensemble 1= 99999.0
weights:
accepted      49452
rejected      50549
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 105010
sum weights ensemble 2= 130750.0
weights:
accepted      54421
rejected      45580
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 132542
sum weights ensemble 3= 132537.0
weights:
accepted      52849
rejected      47152
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 137930
sum weights ensemble 4= 137929.0
weights:
accepted      49977
rejected      50024
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 147644
sum wei

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/tistools/test/istar_test.py:1333: RuntimeWarning: invalid value encountered in scalar divide
  plocistar[i]["LML"] = [np.sum(select_with_masks(w, [masks["LML"], accmask, ~loadmask]))/(np.sum(select_with_masks(w, [masks["LML"], accmask, ~loadmask]))+np.sum(select_with_masks(w, [masks["LMR"], accmask, ~loadmask]))), np.sum(select_with_masks(w, [masks["LML"], accmask, ~loadmask]))+np.sum(select_with_masks(w, [masks["LMR"], accmask, ~loadmask]))]
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/tistools/test/istar_test.py:1334: RuntimeWarning: invalid value encountered in scalar divide
  plocistar[i]["LMR"] = [np.sum(select_with_masks(w, [masks["LMR"], accmask, ~loadmask]))/(np.sum(select_with_masks(w, [masks["LML"], accmask, ~loadmask]))+np.sum(select_with_masks(w, [masks["LMR"], accmask, ~loadmask]))), np.sum(select_with_masks(w, [masks["LML"], accmask, ~loadmask]))+np.sum(select_with_masks(w, [masks["LMR"], accmask, ~loadmask]))

Weights of the different paths:
wRMR = 4444
wRML = 28605
wLMR = 29562
wLML = 37389
Local crossing probabilities:
pRMR = 0.1344670035401979
pRML = 0.8655329964598021
pLMR = 0.44154680288569254
pLML = 0.5584531971143075
Local crossing probabilities:
p2R = 0.34006
p2L = 0.65994


Ensemble 4 ([3*]):
    --- LMR/LML ---
REPPTIS pLMR approx: 0.4491001320269613 (vs. REPPTIS = 0.44154680288569254) with # weights = 129519.0
REPPTIS pLML fw approx: 0.5508998679730387 (vs. REPPTIS = 0.5584531971143075) with # weights = 129519.0
    START TURNS:
     -> Conditional pLMR/pLML with start turn at interface 0: pLML=0.5505147097246172 <-> pL0MR 0.44948529027538275 with sum 1.0 
    with # weights 95685.0
     -> Conditional pLMR/pLML with start turn at interface 1: pLML=0.5593957101118792 <-> pL1MR 0.44060428988812084 with sum 1.0 
    with # weights 14033.0
     -> Conditional pLMR/pLML with start turn at interface 2: pLML=0.5467400636331499 <-> pL2MR 0.45325993636685014 with sum 1.0 
    with # weigh

In [49]:
X, wpath, W = display_data(pathensembles, interfaces, len(interfaces))

----------
ENSEMBLE [0-] | ID 0
----------
weights:
accepted      59613
rejected      40388
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 100001
weights:
accepted      59613
rejected      40388
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 100001
sum weights ensemble 0= 132537.0
1a. Raw data: unweighted C matrices
C[0] = 
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
1b. Raw data: unwe

/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/tistools/test/istar_test.py:1644: RuntimeWarning: invalid value encountered in divide
  frac_unw = C[i]/np.sum(C[i])
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/tistools/test/istar_test.py:1645: RuntimeWarning: invalid value encountered in divide
  frac_w = X[i]/np.sum(X[i])
/mnt/0bf0c339-34bb-4500-a5fb-f3c2a863de29/DATA/APPTIS/tistools/test/istar_test.py:1649: RuntimeWarning: invalid value encountered in divide
  tr_diff = abs(X[i]-X[i].T) / ((X[i]+X[i].T)/2.0)


weights:
accepted      49452
rejected      50549
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 105010
sum weights ensemble 2= nan
1a. Raw data: unweighted C matrices
C[2] = 
array([[0.0000e+00, 1.3911e+04, 7.3220e+03, 4.6670e+03, 2.3110e+03,
        1.1900e+03, 2.5300e+02, 2.8000e+01, 2.8000e+01, 1.4000e+01,
        2.2000e+01],
       [1.3911e+04, 0.0000e+00, 8.4700e+02, 5.8600e+02, 3.1200e+02,
        1.4900e+02, 2.6000e+01, 1.0000e+00, 2.0000e+00, 0.0000e+00,
        9.0000e+00],
       [7.2720e+03, 8.7800e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [4.6270e+03, 5.6700e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.4130e+03, 2.7700e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00],
    

In [50]:
q_k, q_tot = memory_analysis(wpath, True)

--------------------
ENSEMBLE [0*] | ID 1
--------------------
==== L -> R ====
-> interface 0 is reached, probability to reach interface 1:
    - START at interface 0: 0.05578841788523465 | # weights: 189430.0
-> interface 1 is reached, probability to reach interface 2:
    - START at interface 0: 0.5999242997728993 | # weights: 10568.0
-> interface 2 is reached, probability to reach interface 3:
    - START at interface 0: 0.5430599369085174 | # weights: 6340.0
-> interface 3 is reached, probability to reach interface 4:
    - START at interface 0: 0.44205634620970086 | # weights: 3443.0
-> interface 4 is reached, probability to reach interface 5:
    - START at interface 0: 0.4159001314060447 | # weights: 1522.0
-> interface 5 is reached, probability to reach interface 6:
    - START at interface 0: 0.2717219589257504 | # weights: 633.0
-> interface 6 is reached, probability to reach interface 7:
    - START at interface 0: 0.3081395348837209 | # weights: 172.0
-> interface 7 is rea

In [51]:
ploc_memory(pathensembles, interfaces, trr=True)

Weights of the different paths:
wRMR = 99998
wRML = 0
wLMR = 0
wLML = 0
Local crossing probabilities:
pRMR = 1.0
pRML = 0.0
pLMR = nan
pLML = nan
Local crossing probabilities:
p2R = 1.0
p2L = 0.0
weights:
accepted      59613
rejected      40388
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 100001
sum weights ensemble 0= 0.0
weights:
accepted      68968
rejected      31033
omitted       0
total trajs   100001
total MC weights 100001
total MC + HA weights 100001
sum weights ensemble 1= 99999.0
0.0 [0. 0.] 1 0
q:  [[1.         0.05578842]
 [0.         0.        ]]
p:  [[0.94421158 0.05578842]
 [0.         0.        ]]
Local crossing probabilities computed
Weights of the different paths:
wRMR = 0
wRML = 5182
wLMR = 5386
wLML = 89431
Local crossing probabilities:
pRMR = 0.0
pRML = 1.0
pLMR = 0.056804159591634415
pLML = 0.9431958404083656
Local crossing probabilities:
p2R = 0.05386053860538605
p2L = 0.9461394613946139
weights:
accepted      59613
rejected

{'mlst': [1.0,
  0.056804159591634415,
  0.021388019880198804,
  0.010908604553196751,
  0.005591697648448247,
  0.003205665020975919,
  0.0019905040734089397,
  0.0013937351317032127,
  0.0010752551041070932,
  0.0008332124900018845,
  0.000657564781531423],
 'apptis': [1.0,
  0.05578841788523465,
  0.03198366179125235,
  0.018400440245015286,
  0.009553856163908053,
  0.0044947456395435645,
  0.0015298472051736198,
  0.0009861715424868387,
  0.0009073365644977552,
  0.00081956209451956,
  0.000754634224661866],
 'repptis': [1.0,
  0.056804159591634415,
  0.032312997136678026,
  0.018071661236763906,
  0.009174080897093834,
  0.004218791762213937,
  0.0019851528367788125,
  0.0015222409020198072,
  0.001318390278132916,
  0.0011178599945945473,
  0.0009802371708142466]}

In [11]:
# make_plot_trajs("paths002test", "./002", interfaces, 5)

In [12]:
# if 'repptis' in fol:
#     plot_rv_repptis(pathensembles, interfaces, 3)
# else:
#     plot_rv_star(pathensembles, interfaces, 1)

In [13]:
# plot_rv_comp(pathensembles, interfaces, 1, 1, (2,4))
# while True:
#     try:
#         plot_rv_comp(pathensembles, interfaces, 1, 1, (2,4))
#         break
#     except:
#         plt.close("all")
#         continue

# Analysis

In [14]:
# Setting path ensemble properties
#==================================
# for i,fol in enumerate(folders):
#     print(i)
#     if i != 1:
#         print("Calculating path lengths.")
#         set_tau_distrib(pathensembles[i])
#     print("Done.")
#     #else:
#         #TODO problem with ...
#     if i > 1:
#         print("Calculating first hitting lengths to middle interface")
#         set_tau_first_hit_M_distrib(pathensembles[i])
#         print("Done.")
#     #else:
#         # TODO problem with ....

In [15]:
# Analyze the [i*] simulation.
# Analysis output is saved to the data dictionary.
data = {}

data["running"] = {}

# analysis using all data: ["full"]
# data["full"] = get_transition_probs(pathensembles, interfaces)
# pprint(data)
# print("\n\n")
# data["full"] = get_transition_probzz2(pathensembles, interfaces)
w = compute_weight_matrices(pathensembles, interfaces, tr=True)
for i in range(len(pathensembles)):
    print(f"sum weights pe {i}: ",np.sum(w[i]))

data["full"] = get_transition_probzz(w)
# data["full"] = get_transition_probs(w)
data["full1"] = get_simple_probs(w)
# data["full"] = get_summed_probs(pathensembles, interfaces)
pprint(data)

# for i, pe in enumerate(pathensembles):
#     print("doing pathensemble {}".format(i))
#     if i == 0:
#         data[i] = {}
#         continue  # TODO: make [0-] analysis ???

    # masks - TODO not used further on?
    # TODO these functions are duplicate in repptis_analysis
    #masks = get_lmr_masks(pe)
    #loadmask = get_generation_mask(pe, "ld")
    #print("Amount of loads: {}".format(np.sum(loadmask)))
    ##hardloadmask = get_hard_load_mask(loadmask)
    #accmask = get_flag_mask(pe, "ACC")

    # pathtype_cycles
    # pathtypes = ("LML", "LMR", "RML", "RMR")
    # pathtype_cycles = {}
    # for ptype in pathtypes:
    #     pathtype_cycles[ptype] = unwrap_by_weight(
    #             (pe.lmrs == ptype).astype(int), pe.weights)
    
    # # running average analysis: ["running"]
    # data[i] = {}
    # data[i]["running"] = {}
    # data[i]["running"]["plocal"] = {}
    # # you'll still have to hardload select pe.weigths... TODO. # this is comment wouter?
    # for (ptype, p_loc) in zip(pathtypes, 
    #                           running_avg_local_probs(pathtype_cycles, 
    #                                                   pe.weights, tr = False)):
    #     data[i]["running"]["plocal"][ptype] = p_loc

    # analysis using all data: ["full"]
    # plocfull = get_local_probs(pe, tr=False)
    # data[i]["full"] = {}
    # for ptype in pathtypes:
    #     data[i]["full"][ptype] = plocfull[ptype]

    # data[i] have now ["full"] and ["running"]

weights:
accepted      77751
rejected      22249
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 100000
sum weights ensemble 0= 0.0
weights:
accepted      58518
rejected      41482
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 100000
sum weights ensemble 1= 99998.0
weights:
accepted      50666
rejected      49334
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 101416
sum weights ensemble 2= 102339.0
weights:
accepted      47604
rejected      52396
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 124396
sum weights ensemble 3= 124395.0
weights:
accepted      47614
rejected      52386
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 124047
sum weights ensemble 4= 124046.0
weights:
accepted      51103
rejected      48897
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 101326
sum wei

In [16]:

# for i, pe in enumerate(pathensembles):
#     upe = pe.unify_pe()
#     # Pathlength distribution
#     data[i]["pathlengths"] = pathlength_distr(upe)  # these might be used later or not! TODO
        
# #=======================================
# # make figures
# makefigs = True 
# if makefigs:
#     for i, pe in enumerate(pathensembles):     
#         if i == 0:
#             continue
#         # Cross distances distribution
#         L, M, R, lmlpercs, lmllambs, rmrpercs, rmrlambs = cross_dist_distr(pe)
#         fig,ax = plt.subplots()
#         ax.plot(lmllambs, lmlpercs, lw=1, c="g")
#         ax.plot(rmrlambs, rmrpercs, lw=1, c="r")
#         for lamb in (L,M,R):
#             ax.axvline(lamb, color='k', linestyle='--', lw = 0.5)
#         ax.set_xlabel('Cross distance')
#         ax.set_ylabel('Frequency')
#         ax.set_title("Ensemble {}. L = {}, M = {}, R = {}".format(
#             pe.name, L, M, R))
#         ax.set_ylim(0)
#         fig.savefig(f"pathensemble_{i}_crossdist.pdf")
#         plt.close(fig)

#         # Pathlength distribution      
#         for ptype in pathtypes:
#             fig, ax = plt.subplots()
#             ax.plot(data[i]["pathlengths"][ptype]["bin_centers"], 
#                 data[i]["pathlengths"][ptype]["hist"])
#             ax.set_xlabel('Pathlength')
#             ax.set_ylabel('Frequency')
#             ax.set_title(f"{np.sum(data[i]['pathlengths'][ptype]['hist'])} " + \
#                          f"{ptype} paths. ")
#             ax.legend([f"mean = {data[i]['pathlengths'][ptype]['mean']:.2f}, " + \
#                           f"std = {data[i]['pathlengths'][ptype]['std']:.2f}"])
#             fig.savefig(f"pathensemble_{i}_pathlength_{ptype}.pdf")
#             plt.close(fig)

In [17]:
# Make a figure of the global crossing probabilities
# fig, ax = plt.subplots()
# ax.set_yscale("log")
# ax.plot(Pcrossfull, "o", c = "r")

# cosdip meta
# ax.errorbar([i for i in range(7)], Pcrossfull, yerr=[0, 0.004830, Pcrossfull[2]*0.05068988646, Pcrossfull[3]*0.05189862680, Pcrossfull[4]*0.05071184896, Pcrossfull[5]*0.05083284286, Pcrossfull[6]*0.05067963543], fmt="-o", c = "b", ecolor="r", capsize=6)

# cosbump meta
# ax.errorbar([i for i in range(7)], Pcrossfull, yerr=[0, 0.002535, Pcrossfull[2]*0.04393065503, Pcrossfull[3]*0.04910273500, Pcrossfull[4]*0.05239942040, Pcrossfull[5]*0.05789033634, Pcrossfull[6]*0.0614468], fmt="-o", c = "b", ecolor="r", capsize=6)

# 2 cosdips
# ax.errorbar([i for i in range(5)], Pcrossfull, yerr=[0, 0.007239, Pcrossfull[2]*0.0414296, Pcrossfull[3]*0.0445266, Pcrossfull[4]*0.0483538], fmt="-o", c = "b", ecolor="r", capsize=6)

# 3 cosbumps
# ax.errorbar([i for i in range(7)], Pcrossfull, yerr=[0, 0.002295, Pcrossfull[2]*0.0328798, Pcrossfull[3]*0.031594, Pcrossfull[4]*0.031474, Pcrossfull[5]*0.03080392, Pcrossfull[6]*0.0308589], fmt="-o", c = "b", ecolor="r", capsize=6)

# 2 cosbumps
#ax.errorbar([i for i in range(5)], Pcrossfull, yerr=[0, 0.002768, Pcrossfull[2]*0.04440278, Pcrossfull[3]*0.043053, Pcrossfull[4]*0.0463156], fmt="-o", c = "b", ecolor="r", capsize=6)

# flat dt=0.00002 30k cycles
# ax.errorbar([i for i in range(5)], Pcrossfull, yerr=[0, 0.003294, Pcrossfull[2]*0.07640968, Pcrossfull[3]*0.07789262, Pcrossfull[4]*0.0812692], fmt="-o", c = "b", ecolor="r", capsize=6)

# flat 100k cycles
#ax.errorbar([i for i in range(5)], Pcrossfull, yerr=[0, 0.002741, Pcrossfull[2]*0.034092, Pcrossfull[3]*0.033621, Pcrossfull[4]*0.0398], fmt="-o", c = "b", ecolor="r", capsize=6)

# ax.set_xlabel("intf")
# ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
# ax.set_xticks(np.arange(len(interfaces)))
# fig.tight_layout()
# fig.show()
# fig.savefig("Global_probs.pdf")

# print("This should be the same as the repptis_report.pdf value:", Pcrossfull[-1])
# print("which is the case!")
# print("Here, the load immediately disappeared. For a simulation where this is")
# print("not the case, the above code should be adapted a little bit.")

# Now work with MSM

In [18]:
from tistools import construct_M
from tistools import global_cross_prob
from tistools import mfpt_to_first_last_state

from tistools import create_labels_states

In [19]:
def print_vector(g, states=None):
    if states is None:
        for i in range(len(g)):
            print("state", i, g[i])
    else:
        for i in range(len(g)):
            print("state", states[i], g[i][0])

In [20]:
print(interfaces)
N = len(interfaces)
assert N >= 3
NS = 2*N
print("N", N)
print("NS", NS)

#labels2 = ["0+- LML","0+- LMR","0+- RML","1+- LML","1+- LMR",
#           "1+- RML", "1+- RMR", "2+- LML", "2+- LMR",
#           "2+- RML", "2+- RMR", "3+- LML", "3+- LMR",]
labels1, labels2 = create_labels_states(N)
print(labels1, labels2)

[-0.35, -0.25, -0.12, 0.12, 0.25, 0.35]
N 6
NS 12
['0-     ', 'B      '] ['0+- LML', '0+- LMR', '0+- RML', '1+- LML', '1+- LMR', '1+- RML', '1+- RMR', '2+- LML', '2+- LMR', '2+- RML', '2+- RMR', '3+- LML', '3+- LMR', '3+- RML', '3+- RMR', '4+- LML', '4+- LMR']


In [21]:
def construct_M_istar(P, NS, N):
    """Construct transition matrix M"""
    # N -- number of interfaces
    # NS -- dimension of MSM, 4*N-5 when N>=4
    # P -- ndarray of probabilities for paths between end turns
    
    # assert N>=3
    assert N==P.shape[0]
    assert N==P.shape[1]
    assert NS==max(4, 2*N)

    # construct transition matrix
    M = np.zeros((NS,NS))
    
    # states [0-] and [0*+-]
    M[0,2] = 1
    M[2,1] = P[0,0]
    M[2,N+1:] = P[0, 1:]
    M[1,0] = 1
    M[-1,0] = 1
    M[N+1:,1] = P[1:, 0]

    # non-sampled paths
    # M[N+1, -1] = 1

    for i in range(1,N):
        #print("starting from state i",i)
        M[2+i, N+i:2*N] = P[i,i:]
        M[N+i, 3:2+i] = P[i, 1:i]
    

    # for i in range(NS):
    #     if np.sum(M[i]) > 0:
    #         M[i] = M[i]/np.sum(M[i])
    #     else:
    #         M[i] = 0 
       
    # non-sampled paths
    if not M[N, -1] >= 0:
        M[N, -1] = 0
    M[N+1,1] = 1
    # return np.delete(np.delete(M, N, 0), N, 1)
    return M

In [22]:
p_ini = data["full"]
print("p matrix: ", p_ini)
p_ini[-2][-1] = 1
print("sum rows of p:")
for i in range(p_ini.shape[0]):
    print(np.sum(p_ini[i][:i]), np.sum(p_ini[i][i:]))
M = construct_M_istar(p_ini, NS, N)
M1 = construct_M_istar(data["full1"], NS, N)

# for r in range(M.shape[0]):
#     if np.sum(M[r]) != 0:
#         M[r] /= np.sum(M[r])
#Local crossing probabilities:
#pRMR = 0.34205627942625644.  #ppps
#pRML = 0.6579437205737436.   #ppms
#pLMR = 0.25316455696202533.  #pmps
#pLML = 0.7468354430379747.   #pmms

p matrix:  [[0.17184958 0.01425095 0.0166028  0.23725263 0.01270092 0.54734313]
 [1.         0.         0.04947204 0.41257993 0.01562465 0.52232338]
 [0.93134328 0.06865672 0.         0.55631241 0.01784561 0.42584198]
 [0.42251583 0.01829809 0.55918607 0.         0.06864407 0.93135593]
 [0.5227493  0.01601356 0.4129174  0.04831975 0.         1.        ]
 [0.         0.         0.         0.         0.         0.        ]]
sum rows of p:
0.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
1.0 1.0
0.0 0.0


In [23]:
print("M")
print("shape", M.shape)
print("sum prob in rows", np.sum(M,axis=1))
print(M)
print(M1)
# row 8, 10, 12, 14. # counting starts from 0   not okay!!!!

M
shape (12, 12)
sum prob in rows [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.17184958 0.         0.         0.         0.
  0.         0.01425095 0.0166028  0.23725263 0.01270092 0.54734313]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.04947204 0.41257993 0.01562465 0.52232338]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.55631241 0.01784561 0.42584198]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.06864407 0.93135593]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         1.        ]
 [0.         1.         0

# Look at this Markov model

In [24]:
#import numpy.linalg
vals, vecs = np.linalg.eig(M)
print(vals)
vals, vecs = np.linalg.eig(M.T)
print(vals)

[ 0.00000000e+00+0.j          1.00000000e+00+0.j
 -3.98050828e-01+0.76429424j -3.98050828e-01-0.76429424j
 -5.70593179e-01+0.j          4.21119532e-01+0.j
 -7.19262792e-02+0.j         -5.76863142e-02+0.j
 -2.64103971e-02+0.j         -1.61928882e-16+0.j
  5.59224219e-02+0.j          4.56758706e-02+0.j        ]
[ 1.00000000e+00+0.j         -3.98050828e-01+0.76429424j
 -3.98050828e-01-0.76429424j -5.70593179e-01+0.j
  4.21119532e-01+0.j          5.59224219e-02+0.j
  4.56758706e-02+0.j         -7.19262792e-02+0.j
 -5.76863142e-02+0.j         -2.64103971e-02+0.j
  1.03682274e-18+0.j          0.00000000e+00+0.j        ]


In [25]:
print("what if chain propagates")
print("A[0,:]")
# check stationary behavior
A = M
for n in range(10):
    A = np.dot(A,M)
    #print(A)
    print(A[0,:])
    print(np.sum(A[2,:]))  # is 1 indeed

what if chain propagates
A[0,:]
[0.         0.17184958 0.         0.         0.         0.
 0.         0.01425095 0.0166028  0.23725263 0.01270092 0.54734313]
1.0
[7.19192702e-01 1.36596244e-01 0.00000000e+00 5.68455193e-03
 1.37912796e-01 6.13705334e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
1.0000000000000002
[1.36596244e-01 0.00000000e+00 7.19192702e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 2.81226366e-04 7.90679322e-02 2.59208384e-03 6.22698110e-02]
1.0000000000000002
[6.22698110e-02 1.58617343e-01 1.36596244e-01 1.50760909e-03
 4.52840029e-02 1.25248831e-04 0.00000000e+00 1.02491765e-02
 1.19406143e-02 1.70630358e-01 9.13441065e-03 3.93645181e-01]
1.0000000000000002
[5.52262524e-01 1.21713029e-01 6.22698110e-02 4.08828826e-03
 9.91858766e-02 4.41372398e-04 0.00000000e+00 1.94662572e-03
 2.34246494e-03 5.82218799e-02 2.57517224e-03 9.49529556e-02]
1.0000000000000002
[2.16665984e-01 4.0775141

# Pcross with MSM

In [26]:
def global_cross_prob_star(M, doprint=False):
    # probability to arrive in -1 before 0
    # given that you are at 0 now and that you are leaving 0
    # = crossing probability from 0 to -1

    NS = len(M)
    assert NS>2

    # take pieces of transition matrix
    Mp = M[2:-1,2:-1]
    a = np.identity(NS-3)-Mp    # 1-Mp
    # a1 = np.linalg.inv(a)       # (1-Mp)^(-1)  --> bad practice!

    # other pieces
    D = M[2:-1, np.array([0,-1])]
    E = M[np.array([0,-1]), 2:-1]
    M11 = M[np.array([0,-1]),np.array([0,-1])]

    # compute Z vector
    z1 = np.array([[0],[1]])
    # z2 = np.dot(a1,np.dot(D,z1))
    z2 = np.linalg.solve(a, np.dot(D,z1))

    # compute H vector
    y1 = np.dot(M11,z1) + np.dot(E,z2)
    y2 = np.dot(D,z1) + np.dot(Mp,z2)

    if doprint:
        print("Mp eigenvals")
        vals, vecs = np.linalg.eig(Mp)
        print(vals)
        print("1-Mp eigenvals")
        vals, vecs = np.linalg.eig(a)
        print(vals)
        #print(np.dot(a,a1)  # identity matrix indeed
        print("other pieces M")
        print(D)
        print(E)
        print(M11)
        print("vector z1,z2")
        print(z1)
        print(z2)
        print("vector y1,y2")
        print(y1)
        print(y2)
        print("check", np.sum((y2-z2)**2))  # 0, so z2 and y2 indeed the same
    return z1, z2, y1, y2

In [27]:
# global crossing prob
z1, z2, y1, y2 = global_cross_prob_star(M, True)
print("Z")
print_vector(z1, labels1)
print_vector(z2, labels2)
print("Y")
print_vector(y1, labels1)
print_vector(y2, labels2)
print("global crossing prob", y1[0])

Mp eigenvals
[ 0.         -0.57131864  0.57131864 -0.05842661 -0.05608332  0.05842661
  0.05608332  0.          0.        ]
1-Mp eigenvals
[1.         0.42868136 1.57131864 0.94157339 0.94391668 1.05608332
 1.05842661 1.         1.        ]
other pieces M
[[0.         0.54734313]
 [0.         0.52232338]
 [0.         0.42584198]
 [0.         0.93135593]
 [0.         1.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]
 [0.         0.        ]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[0. 0.]
vector z1,z2
[[0]
 [1]]
[[0.63958266]
 [0.68162447]
 [0.63648735]
 [0.953308  ]
 [1.        ]
 [0.        ]
 [0.0467981 ]
 [0.36838729]
 [0.31979553]]
vector y1,y2
[[0.63958266]
 [0.        ]]
[[0.63958266]
 [0.68162447]
 [0.63648735]
 [0.953308  ]
 [1.        ]
 [0.        ]
 [0.0467981 ]
 [0.36838729]
 [0.31979553]]
check 1.237409989268799e-32
Z
state 0-      0
state B       1
state 0+- LML 0.6395826553421684
state 0+- LMR 0.6816244749591692
s

In [28]:
# P_loc with MSM
plocMSM = np.ones(len(interfaces))
# p2 = [1.0, 0.056804159591634415, 0.032312997136678026, 0.018071661236763906, 0.009174080897093834, 0.004218791762213937, 0.0019851528367788125, 0.0015222409020198072, 0.001318390278132916, 0.0011178599945945473, 0.0009802371708142466]
p2 = [1.0, 0.8272598636455519, 0.8118884810440299, 0.7938415404869816, 0.5663700466714486, 0.5545303803303038]
# p3 = [1.0, 0.062089, 0.035751653, 0.02010748, 0.010213876, 0.004782699, 0.001383816, 0.000568035, 0.000547059, 0.000516629, 0.000493516]
p3 = [1.0, 0.810296, 0.796242226, 0.78048937, 0.636988594, 0.625611341]

for lint in range(2, len(interfaces)+1):
    wi = compute_weight_matrices(pathensembles[:lint], interfaces[:lint], len(interfaces), True)
    pi = get_transition_probzz(wi)
    # pi = get_transition_probs(wi)
    # pi = get_simple_probs(wi)
    Mi = construct_M_istar(pi, max(4, 2*len(interfaces[:lint])), len(interfaces[:lint]))
    z1, z2, y1, y2 = global_cross_prob_star(Mi)
    plocMSM[lint-1] = y1[0][0]
    print(f"ploc till intf {lint-1}: ", y1[0][0])

# Make a figure of the global crossing probabilities
plt.rcParams['text.usetex'] = True
fig, ax = plt.subplots()
ax.set_yscale("log")
ax.plot(plocMSM, "o", c = "r")
ax.errorbar([i for i in range(len(plocMSM))], plocMSM, fmt="-o", c = "b", ecolor="r", capsize=6, label="StapleTIS")
ax.errorbar([i for i in range(len(plocMSM))], p2, fmt="-o", c = "orange", ecolor="r", capsize=6., label="REPPTIS")
ax.errorbar([i for i in range(len(plocMSM))], p3, fmt="-o", c = "r", ecolor="r", capsize=6., label="RETIS")
# ax.set_xlabel(r"Interface index")
ax.set_ylabel(r"$P_A(\lambda_i|\lambda_A)$")
ax.set_xticks(np.arange(len(interfaces)))
fig.tight_layout()
fig.legend()
fig.show()


weights:
accepted      77751
rejected      22249
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 100000
sum weights ensemble 0= 0.0
weights:
accepted      58518
rejected      41482
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 100000
sum weights ensemble 1= 99998.0
0.0 [0. 0.] 1 0
q:  [[1.         0.82815042]
 [0.         0.        ]]
p:  [[0.17184958 0.82815042]
 [0.         0.        ]]
Local crossing probabilities computed
ploc till intf 1:  0.8281504232252875
weights:
accepted      77751
rejected      22249
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 100000
sum weights ensemble 0= 0.0
weights:
accepted      58518
rejected      41482
omitted       0
total trajs   100000
total MC weights 100000
total MC + HA weights 100000
sum weights ensemble 1= 99998.0
weights:
accepted      50666
rejected      49334
omitted       0
total trajs   100000
total MC weights 100000
total MC + H

In [29]:
print(plocMSM)
for pp in plocMSM:
    print(pp)
print("\n\n")
pcrosslocMSM = np.empty(len(plocMSM))

for i in range (len(pcrosslocMSM)):
    pcrosslocMSM[i] = plocMSM[i]/np.prod(pcrosslocMSM[:i])
    print(pcrosslocMSM[i])
    

[1.         0.82815042 0.81389948 0.79838387 0.65067041 0.63958266]
1.0
0.8281504232252875
0.8138994771000272
0.7983838685343494
0.6506704124653093
0.6395826553421684



1.0
0.8281504232252875
0.9827918386254527
0.9809367016416317
0.8149844180341366
0.9829594877671927


In [54]:
# Plot network
def plot_network(P, figname, source=None, target=None):
    """
    Network plots
    =============
    
    We demonstrate different kinds of network plots based on :meth:`plots.Network <deeptime.plots.Network>`.
    In particular:
    
        * plotting a Markov state model where the state sizes depend on the stationary distribution and edges are scaled
          according to jump probabilities (:meth:`deeptime.plots.plot_markov_model`)
        * plotting the gross flux, in accordance to edge widths and colored according to the forward committor
          (:meth:`deeptime.plots.Network`).
    """
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import networkx as nx
    
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    
    from deeptime.markov.msm import MarkovStateModel
    from deeptime.plots import plot_markov_model, Network, plot_flux
    
    if source is None:
        source = 0
    elif target is None:
        target = len(P)-1

    f1, axes1 = plt.subplots(figsize=(16, 12))
    axes1.set_aspect('equal')
    f2, axes2 = plt.subplots(figsize=(16, 12))
    axes2.set_aspect('equal')
    f3, axes3 = plt.subplots(figsize=(16, 12))
    axes3.set_aspect('equal')

    ax = axes1
    ax.set_title('Plotting the Markov model')
    plot_markov_model(P, ax=ax)
    
    ax = axes2
    ax.set_title('Plotting the gross flux')
    flux = MarkovStateModel(P).reactive_flux(source_states=source,
                                             target_states=target)
    positions = nx.multipartite_layout(nx.from_numpy_array(flux.gross_flux))
    #cmap = mpl.cm.get_cmap('coolwarm')
    cmap = mpl.colormaps['coolwarm'] #m.get_cmap('coolwarm')
    network = Network(flux.gross_flux, positions, edge_curvature=2.,
                      state_colors=flux.forward_committor, cmap=cmap)
    network.plot(ax=ax)
    norm = mpl.colors.Normalize(vmin=np.min(flux.forward_committor), vmax=np.max(flux.forward_committor))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    f1.colorbar(mpl.cm.ScalarMappable(norm, cmap), cax=cax)
    
    ax = axes3
    ax.set_title('Plotting the net flux')
    ax.get_yaxis().set_visible(False)
    plot_flux(flux, attribute_to_plot='net_flux', ax=ax)
    
    plt.show()
    # f.savefig(figname)

In [ ]:
A = [0]
B = [len(M)-1]

plot_network(M, "fig_try-plot-network-M.png", source=A, target=B)
# from deeptime.plots import plot_markov_model, Network, plot_flux
# plot_markov_model(M)

# Error analysis (recursive block analysis)

In [30]:
# PyRETIS code for block error analysis
 
def block_error(data, maxblock=None, blockskip=1):
    """
    Perform block error analysis to estimate the standard deviation in the input data.

    Parameters
    ----------
    data : numpy.array
        The data to analyze.
    maxblock : int, optional
        Maximum block length to consider. Defaults to half the length of the input data.
    blockskip : int, optional
        Skip factor for block lengths. Defaults to 1 (all block lengths considered).

    Returns
    -------
    blocklen : numpy.array
        Array of block lengths considered.
    block_avg : numpy.array
        Block averages as a function of block length.
    block_err : numpy.array
        Standard error estimates as a function of block length.
    block_err_avg : float
        Average error estimate for block lengths greater than maxblock//2.
    """
    n = len(data)
    maxblock = min(maxblock or n // 2, n // 2)
    
    blocklen = np.arange(1, maxblock + 1, blockskip, dtype=np.int_)
    n_blocks = len(blocklen)
    
    block = np.zeros(n_blocks)
    nblock = np.zeros(n_blocks)
    block_avg = np.zeros(n_blocks)
    block_var = np.zeros(n_blocks)

    for i, val in enumerate(data):
        block += val
        full_blocks = (i + 1) % blocklen == 0
        block[full_blocks] /= blocklen[full_blocks]
        nblock[full_blocks] += 1
        deltas = block[full_blocks] - block_avg[full_blocks]
        block_avg[full_blocks] += deltas / nblock[full_blocks]
        block_var[full_blocks] += deltas * (block[full_blocks] - block_avg[full_blocks])
        block[full_blocks] = 0.0

    block_var /= (nblock - 1)
    block_err = np.sqrt(block_var / nblock)
    
    large_blocks = blocklen > maxblock // 2
    block_err_avg = np.mean(block_err[large_blocks])
    
    return blocklen, block_avg, block_err, block_err_avg, maxblock, n//maxblock


def block_error_corr(data, maxblock=None, blockskip=1):
    """
    Run block error analysis and calculate correlation length estimates.

    Parameters
    ----------
    data : numpy.array
        Data to analyze.
    maxblock : int, optional
        Maximum block length to consider. Defaults to half the length of the input data.
    blockskip : int, optional
        Skip factor for block lengths. Defaults to 1 (all block lengths considered).

    Returns
    -------
    blen : numpy.array
        Block lengths considered.
    berr : numpy.array
        Error estimates as a function of block length.
    berr_avg : float
        Average error estimate for blocks with length > maxblock // 2.
    rel_err : numpy.array
        Relative error normalized by the overall average as a function of block length.
    avg_rel_err : float
        Average relative error for blocks with length > maxblock // 2.
    ncor : numpy.array
        Estimated correlation length as a function of block length.
    avg_ncor : float
        Average correlation length for blocks with length > maxblock // 2.
    """
    blen, bavg, berr, berr_avg, max_block_size, min_block_number = block_error(data, maxblock=maxblock, blockskip=blockskip)
    rel_err = berr / abs(bavg[0])
    avg_rel_err = berr_avg / abs(bavg[0])
    ncor = (berr / berr[0])**2
    avg_ncor = (berr_avg / berr[0])**2
    
    return blen, berr, berr_avg, rel_err, avg_rel_err, ncor, avg_ncor, max_block_size, min_block_number

def pathensembles_nskip(obj, nskip):
    keys = [
        'cyclenumbers', 'flags', 'generation', 'lambmaxs', 'lambmins',
        'lengths', 'lmrs', 'newpathnumbers', 'pathnumbers',
        'shootlinks', 'weights', 'dirs', 'istar_idx']
    for key in keys:
        attr = getattr(obj, key)
        setattr(obj, key, attr[:nskip])

In [33]:
%%capture
from datetime import datetime
import copy

start_cyc = 1
n_cycle = 100000
nstep = 1

ploc_MSM_stored = []
ploc_repptis_stored = []
# This loops over the npy file and calculates tau from cycle 100 every 10 cycles
for nskip in range(start_cyc, n_cycle, nstep):
    pathensemblesi = copy.deepcopy(pathensembles)
    for i, pe in enumerate(pathensemblesi):
        pathensembles_nskip(pe,nskip)
    # Analysis output is saved to the data dictionary.
    data = {}
    for i, pe in enumerate(pathensemblesi):
        if i == 0:
            data[i] = {}
            continue  #  [0-] is not used for Pcross calculations
        
        # Classify the paths according to their path type.
        pathtypes = ("LML", "LMR", "RML", "RMR")
        pathtype_cycles = {}
        for ptype in pathtypes:
            pathtype_cycles[ptype] = unwrap_by_weight(
                    (pe.lmrs == ptype).astype(int), pe.weights)
        
        data[i] = {}
        plocfull = get_local_probs(pe, tr=False)
        data[i]["full"] = {}
        for ptype in pathtypes:
            data[i]["full"][ptype] = plocfull[ptype]

    psfull = []
    for i in range(1, len(pathensemblesi)):   # do not use the 0- ensemble
        psfull.append({"LMR": data[i]["full"]["LMR"], 
                "RML": data[i]["full"]["RML"], 
                "RMR": data[i]["full"]["RMR"],
                "LML": data[i]["full"]["LML"]})

    Pminfull, Pplusfull, Pcrossfull = get_globall_probs(psfull)

    ploc_repptis_stored.append(Pcrossfull)
    
    N = len(interfaces)
    NS = 2*N

    wi = compute_weight_matrices(pathensemblesi, interfaces, len(interfaces), True)
    pi = get_transition_probzz(wi)
    # pi = get_transition_probs(wi)
    # pi = get_simple_probs(wi)
    M = construct_M_istar(pi, max(4, 2*len(interfaces)), len(interfaces))

    plocMSM = np.ones(len(interfaces))
    for lint in range(1, len(interfaces)):
        # Mi = M[:min(NS, 1+2*lint), :min(NS, 1+2*lint)]
        Mi = M[np.r_[0:2+lint, 1+N:N+lint, -1]]
        Mi = Mi[:, np.r_[0:2+lint, 1+N:N+lint+1]]
        Msum = M[np.r_[0:2+lint, 1+N:N+lint+1]]
        Mi[:, -1] = np.sum(Msum[:, N+lint:], axis=1)
        
        # wi2 = compute_weight_matrices(pathensemblesi[:lint+1], interfaces[:lint+1], len(interfaces), True)
        # pi2 = get_transition_probzz(wi2)
        # # pi = get_transition_probs(wi)
        # # pi = get_simple_probs(wi)
        # Mi2 = construct_M_istar(pi2, max(4, 2*len(interfaces[:lint+1])), len(interfaces[:lint+1]))

        # print(Mi == Mi2)

        z1, z2, y1, y2 = global_cross_prob_star(Mi)
        plocMSM[lint] = y1[0][0]
    # print(f"plocs: ", plocMSM)

    pcrosslocMSM = np.empty(len(plocMSM))

    for i in range (len(pcrosslocMSM)):
        pcrosslocMSM[i] = plocMSM[i]/np.prod(pcrosslocMSM[:i])
    # print(pcrosslocMSM)

    ploc_MSM_stored.append(plocMSM)

    # for i,fol in enumerate(folders):
    #     set_tau_distrib(pathensemblesi[i])
    #     if True:
    #         set_tau_first_hit_M_distrib(pathensemblesi[i])

    # Compute taus for pathlength analysis
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"{nskip:5d} cycles, Plocs {plocMSM}")

    # Not sure if we need this, need to check later
    del data
    del Mi, wi, pi, z1, z2, y1, y2,N, NS
    del Pcrossfull, Pminfull, Pplusfull
    del plocMSM, pcrosslocMSM

np.save('plocMSM_vs_cycle_interval_1_full.npy', ploc_MSM_stored)
np.save('plocREPPTIS_vs_cycle_interval_1_full.npy', ploc_repptis_stored)

In [ ]:
# Adapted AG, Jan 17, 2025

#----------------
# part Titus
#----------------

# SETTINGS

minblocks = 5 #the minimal number of blocks
filerunav = "plocMSM_vs_cycle_interval_1_full.npy"

# FUNCTIONS

def get_second_column(file):
    arr = []
    with open(file, "r") as f:
        for line in f:
            values = line.strip().split()
            arr.append(float(values[1]))
    return arr
 
def get_first_column(file):
    arr = []
    with open(file, "r") as f:
        for line in f:
            values = line.strip().split()
            arr.append(float(values[0]))
    return arr
 
def rec_blocks(r):
    # no longer used
    """Compute the vector b, which is ... """
    b = np.zeros(len(r))
    for i in range(len(r)):
        if i == 0:
            b[i] = r[i]
        else:
            b[i] = (i+1) * r[i] - i * r[i - 1]
    return b
 
def rec_blocks_from_runav(runav, n):
    # see paper Vervust et al. PyRETIS3, 2024
    # paper: select values k[j*m], indexing starts at 1
    # here: runav, python indexing, so start index n-1 (end of first block)
    # collect them in runav_red
    assert n>0   # block size
    runav_red = runav[n-1::n]
    nb = len(runav_red)  # number of blocks
 
    # array b with average of each block
    b = np.zeros(nb)
    for i in range(nb):
        if i == 0:
            b[i] = runav_red[i]
        else:
            b[i] = (i+1) * runav_red[i] - i * runav_red[i - 1]
    return b
 
def compute_absolute_error(runavfull, bestav, n):
    # compute average in each block
    assert n>0
    blocks = rec_blocks_from_runav(runavfull, n)
    # compute standard deviation between blocks
    sum_qudiff = np.sum((blocks-bestav)**2)
    nb = len(blocks)
    assert nb>1
    Aerr2 = sum_qudiff/(nb*(nb-1))
    Aerr = np.sqrt(Aerr2)  #esitimate of absolute error
    return Aerr
 
 
# GET STARTED
 
# runavfull = get_second_column(filerunav) #all the running average data in an array
runavfull = np.load(filerunav)
runavfull = runavfull[5:,:]
maxbll = int(len(runavfull)/minblocks) #maximum block length
bestav = runavfull[-1] #most accurate average we have
print("maximum block length = ", maxbll)
 
# loop over block size
sizes = np.arange(1,maxbll+1)
if len(np.shape(runavfull)) == 1:
    rel_errors = np.zeros(len(sizes)) if len(np.shape(runavfull)) == 1 else np.zeros((len(sizes), np.shape(runavfull)[1]))
    for i, n in enumerate(sizes):
        # n is block size
        Aerr = compute_absolute_error(runavfull, bestav, n)
        Rerr = Aerr/bestav
        rel_errors[i] = Rerr
    
    second_half = rel_errors[len(rel_errors)//2:]
    half_av_err = np.mean(second_half)
    print("estimated relative error=", half_av_err)
    Nstatineff=(half_av_err/rel_errors[0])**2
    print("statistical inefficiency=", Nstatineff)
    
    plt.figure()
    plt.plot(sizes, rel_errors)
    plt.axhline(y=half_av_err, color='r', linestyle='--')
    plt.xlabel('Block Length')
    plt.ylabel('Relative Error')
    plt.title('Error Analysis')
    plt.show()
    # plt.savefig("figure_titus.png")

elif len(np.shape(runavfull)) == 2:
    rel_errs = np.empty(len(interfaces))
    for loc in range(len(interfaces)):
        rel_errors = np.zeros(len(sizes))
        intf_i = runavfull[:,loc]
        bestav = intf_i[-1]
        for i, n in enumerate(sizes):
        # n is block size
            Aerr = compute_absolute_error(intf_i, bestav, n)
            Rerr = Aerr/bestav
            rel_errors[i] = Rerr

        second_half = rel_errors[len(rel_errors)//2:]
        half_av_err = np.mean(second_half)
        rel_errs[loc] = half_av_err
        print("estimated relative error=", half_av_err)
        Nstatineff=(half_av_err/rel_errors[0])**2
        print("statistical inefficiency=", Nstatineff)
        
        plt.figure()
        plt.plot(sizes, rel_errors)
        plt.axhline(y=half_av_err, color='r', linestyle='--')
        plt.xlabel('Block Length')
        plt.ylabel('Relative Error')
        if loc < len(interfaces)-1:
            plt.title('Error Analysis $p_{local}$'+f' - interface {loc}')
        else:
            plt.title('Error Analysis $P_{cross}$')
        plt.show()
        # plt.savefig("figure_titus.png")

maximum block length =  19998
estimated relative error= 0.0
statistical inefficiency= nan


/tmp/ipykernel_373984/3074760271.py:122: RuntimeWarning: invalid value encountered in scalar divide
  Nstatineff=(half_av_err/rel_errors[0])**2


estimated relative error= 0.002976917749157165
statistical inefficiency= 2.5656294337197516
estimated relative error= 0.0030662076838438436
statistical inefficiency= 2.615462683881789
estimated relative error= 0.0029899464230427066
statistical inefficiency= 2.404594945920509
estimated relative error= 0.003071168142124487
statistical inefficiency= 1.9185859990105758
estimated relative error= 0.0030963938425098325
statistical inefficiency= 1.9185463810508887


IndexError: index 6 is out of bounds for axis 1 with size 6

In [ ]:
stored_values = np.load('plocREPPTIS_vs_cycle_interval_10.npy')

if len(np.shape(stored_values)) == 1:
    stored_values = stored_values[~np.isnan(stored_values)] # remove nans in the beginning
    blen, berr, berr_avg, rel_err, avg_rel_err, ncor, avg_ncor, max_block_size, min_block_number = block_error_corr(stored_values,10)
    print("=" * 60)
    print(indir[57:-9])
    print(f"Total Data Points: {len(stored_values)}, max block size: {max_block_size}, min block number: {min_block_number}")
    print(f"Average Relative Error for blocks > maxblock/2: {avg_rel_err * 100:.1f}%")
    print(f"Average Correlation Length for Large Blocks: {int(avg_ncor)}")
    plt.figure(figsize=(8, 6))
    plt.plot(rel_err, marker='o', linestyle='-', label = indir[57:-9])
    plt.xlabel("Block Size")
    plt.ylabel("Rlative Error")
    plt.title(f"Ave Rel Err (blocks > maxblock/2): {avg_rel_err * 100:.1f}%, Block interval 10 cycles")
    plt.grid(True)
    plt.legend()
    # plt.savefig("Block_Error_Tau.png", dpi=1000, bbox_inches='tight')
elif len(np.shape(stored_values)) == 2:
    rel_errs = np.empty(len(interfaces))
    for loc in range(1,len(interfaces)):
        intf_i = stored_values[:,loc]
        blen, berr, berr_avg, rel_err, avg_rel_err, ncor, avg_ncor, max_block_size, min_block_number = block_error_corr(intf_i[~np.isnan(intf_i)],1000)
        rel_errs[loc] = avg_rel_err

        print("=" * 60)
        print(indir[57:-9])
        print(f"Total Data Points: {len(stored_values)}, max block size: {max_block_size}, min block number: {min_block_number}")
        print(f"Average Relative Error for blocks > maxblock/2: {avg_rel_err * 100:.10f}%")
        print(f"Average Correlation Length for Large Blocks: {int(avg_ncor)}")
        plt.figure(figsize=(8, 6))
        plt.plot(rel_err, linestyle='-', label = indir[57:-9])
        plt.xlabel("Block Size")
        plt.ylabel("Rlative Error")
        plt.title(f"Ave Rel Err (blocks > maxblock/2): {avg_rel_err * 100:.1f}%, Block interval 10 cycles")
        plt.grid(True)
        plt.legend()
        # plt.savefig("Block_Error_Tau.png", dpi=1000, bbox_inches='tight')

# Bootstrap 👢 analysis

In [ ]:
from tistools import bootstrap_istar_analysis

In [ ]:
from tistools import find_closest_number_lte
def bootstrap_istar_analysis2(pathensembles, interfaces, nN=10, nB=1000):
    N = len(interfaces)
    NS = max(3, 2*len(interfaces))
    data = {}
    # for each pathensemble, we save the indices of accepted cycle numbers in a
    # dictionary, because we will use this a lot. We do not accept load cycles, 
    # so if a load cycle is sampled, we will just not use it. 
    pathcycle_ids = {}
    for i, pe in enumerate(pathensembles):
        loadmask = get_generation_mask(pe, "load")
        accmask = get_flag_mask(pe, "ACC")
        pathcycle_ids[i] = select_with_masks(pe.cyclenumbers,
                                             [accmask, ~loadmask])
    for Bcycle in np.arange((pathensembles[0].cyclenumbers)[-1]//nN,
                            (pathensembles[0].cyclenumbers)[-1],
                            ((pathensembles[0].cyclenumbers)[-1]//nN)):
        logger.info(f"Doing bootstrap analysis for cycle {Bcycle}")
        # We produce a list of the data for each timeslice
        ts_data = {}
        for j in range(nB):
            if j % 100 == 0:
                logger.info(f"Doing bootstrap sample {j}")
            # A. Select cycle numbers randomly within the timeslice [1, Bcycle],
            #    using replacement. We start from one to discard the initial
            #    load cycle.
            cycle_ids = np.random.choice(np.arange(start=1,stop=Bcycle), 
                                         Bcycle, replace=True)
            # Store the data for each pathensemble in a dictionary
            boot_data = {}
            boot_pes = []
            for i, pe in enumerate(pathensembles):
                if pe.in_zero_minus:
                    logger.info(f"Passing pathensemble {i} because this is "+\
                                f"the zero minus ensemble: {pe.name}")
                    pass
                # map the cycle numbers to the indices of accepted cycles
                boot_cycle_ids = find_closest_number_lte(cycle_ids,
                                                         pathcycle_ids[i])
                # sample the pathensemble at the given cycle indices
                boot_pes.append(pe.sample_pe(boot_cycle_ids))
                #boot_data[i]['pe'] = boot_pe
            # B. get the transition crossing probabilities
            boot_data['w_path'] = compute_weight_matrices(boot_pes, interfaces)
            boot_data['p'] = get_transition_probzz(boot_data['w_path'])
            # C. Calculate the global crossing probabilities 
            Mj = construct_M_istar(boot_data['p'], NS, N)
            z1, z2, y1, y2 = global_cross_prob_star(Mj)
            boot_data['z1'] = z1
            boot_data['z2'] = z2
            boot_data['y1'] = y1
            boot_data['y2'] = y2
            ts_data[j] = boot_data
        # save the boot_data for this timeslice 
        data[Bcycle] = {}
        data[Bcycle]['data'] = ts_data
        # D. Calculate the mean and std of the local crossing probabilities for
        #    each pathensemble, and the mean and std of the global crossing for 
        #    each bootstrap sample.
        ts_stats = {}
        # first the local crossing probabilities
        for attr in ['p', 'z1', 'z2', 'y1', 'y2']:
            ts_stats[attr] = {}
            ts_stats[attr]['mean'] = \
                np.mean(np.array([ts_data[j][attr]
                                  for j in ts_data.keys()]),axis=0)
            ts_stats[attr]['std'] = \
                np.std(np.array([ts_data[j][attr]
                                 for j in ts_data.keys()]),axis=0)
            
        data[Bcycle]['stats'] = ts_stats

    return data

In [ ]:
bs_data = bootstrap_istar_analysis2(pathensembles, interfaces)

print(bs_data)

# Collecting times

In [ ]:
#TODO I need data too?

In [ ]:
def collect_tau(pathensembles, data):
    # pathensembles -- list of pathensemble instances
    
    print("Collect tau")
    
    # average path lengths
    taumm = np.zeros(len(pathensembles))
    taump = np.zeros(len(pathensembles))
    taupm = np.zeros(len(pathensembles))
    taupp = np.zeros(len(pathensembles))
    
    # for [1+-] and higher (i>=2)
    for i in range(2,len(pathensembles)):
        print("ensemble", i, pathensembles[i].name)
        taumm[i] = pathensembles[i].tauavg['LML']-2
        taump[i] = pathensembles[i].tauavg['LMR']-2
        taupm[i] = pathensembles[i].tauavg['RML']-2
        taupp[i] = pathensembles[i].tauavg['RMR']-2

    # for [0-] (i=0)
    print("ensemble", 0, pathensembles[0].name)
    if pe.has_zero_minus_one:
        # TODO pieces missing
        taumm[0] = pathensembles[0].tauavg['LML']-2
        taump[0] = pathensembles[0].tauavg['LMR']-2
        taupm[0] = pathensembles[0].tauavg['RML']-2
        taupp[0] = pathensembles[0].tauavg['RMR']-2
    else:
        taupp[0] = pathensembles[0].tauavg['RMR']-2

    # for [0+-] (i=1)
    print("ensemble", 1, pathensembles[1].name)
    taumm[1] = data[1]["pathlengths"]["LML"]["mean"] - 2
    taump[1] = data[1]["pathlengths"]["LMR"]["mean"] - 2
    taupm[1] = data[1]["pathlengths"]["RML"]["mean"] - 2
    #taupp[1] = data[1]["pathlengths"]["RMR"]["mean"] - 2

    return taumm, taump, taupm, taupp

# this is the same:
# 1) after:
#   data[2]["pathlengths"] = pathlength_distr(upe)  # use correct upe!
#   This gives the whole distrib, mean, std, etc
#   print(data[2]["pathlengths"]["RMR"]["mean"])
# 2) after:
#   set_tau_distrib(pathensembles[2])
#   print(pathensembles[2].tauavg['RMR'])

# TODO for [0-]
# likely not okay yet, what about L*L etc??????
# There are paths missing TODO!!!!!!!! when lambda-1

In [ ]:
# TODO fix +-1 issues!!!!!!
# TODO tau1 is not fool proof if you have too many phase points!!

def collect_tau1(pathensembles, data):
    # average path lengths, but only the part before the 1st crossing
    #---------------------------
    print("Collect tau1")
    taumm1 = np.zeros(len(pathensembles))
    taump1 = np.zeros(len(pathensembles))
    taupm1 = np.zeros(len(pathensembles))
    taupp1 = np.zeros(len(pathensembles))
    
    # for [1+-] and higher (i>=2)
    for i in range(2,len(pathensembles)):
        taumm1[i] = pathensembles[i].tau1avg['LML'] - 1
        taump1[i] = pathensembles[i].tau1avg['LMR'] - 1
        taupm1[i] = pathensembles[i].tau1avg['RML'] - 1
        taupp1[i] = pathensembles[i].tau1avg['RMR'] - 1
    # for [0-] (i=0) -> just 0
    # for [0+-] (i=1) LML -> just 0
    # for [0+-] (i=1) LMR -> just 0
    # for [0+-] (i=1) RML
    taupm1[1] = data[1]["pathlengths"]["RML"]["mean"] - 2
    
    return taumm1, taump1, taupm1, taupp1

def collect_tau2(pathensembles, data):
    # average path lengths, but only the part after the last crossing
    #---------------------------
    print("Collect tau2")
    taumm2 = np.zeros(len(pathensembles))
    taump2 = np.zeros(len(pathensembles))
    taupm2 = np.zeros(len(pathensembles))
    taupp2 = np.zeros(len(pathensembles))
    
    # for [1+-] and higher (i>=2)
    for i in range(2,len(pathensembles)):
        taumm2[i] = pathensembles[i].tau2avg['LML'] - 1
        taump2[i] = pathensembles[i].tau2avg['LMR'] - 1
        taupm2[i] = pathensembles[i].tau2avg['RML'] - 1
        taupp2[i] = pathensembles[i].tau2avg['RMR'] - 1
    # for [0-] (i=0) -> just 0
    # for [0+-] (i=1) LML -> just 0
    # for [0+-] (i=1) LMR
    taump2[1] = data[1]["pathlengths"]["LMR"]["mean"] - 2
    # for [0+-] (i=1) RML -> just 0
    
    return taumm2, taump2, taupm2, taupp2

def collect_taum(pathensembles, data):
    # average path lengths, but only the part between first/last crossing
    #---------------------------
    print("Collect taum")
    taumm_m = np.zeros(len(pathensembles))
    taump_m = np.zeros(len(pathensembles))
    taupm_m = np.zeros(len(pathensembles))
    taupp_m = np.zeros(len(pathensembles))
    
    # for [1+-] and higher (i>=2)
    for i in range(2,len(pathensembles)):
        #print("ensemble", i, pathensembles[i].name)
        taumm_m[i] = pathensembles[i].tauavg['LML'] \
                   - pathensembles[i].tau1avg['LML'] \
                   - pathensembles[i].tau2avg['LML']
        taump_m[i] = pathensembles[i].tauavg['LMR'] \
                   - pathensembles[i].tau1avg['LMR'] \
                   - pathensembles[i].tau2avg['LMR']
        taupm_m[i] = pathensembles[i].tauavg['RML'] \
                   - pathensembles[i].tau1avg['RML'] \
                   - pathensembles[i].tau2avg['RML']
        taupp_m[i] = pathensembles[i].tauavg['RMR'] \
                   - pathensembles[i].tau1avg['RMR'] \
                   - pathensembles[i].tau2avg['RMR']

    # for [0-] (i=0)
    if pe.has_zero_minus_one:
        # TODO there are more paths!!!
        taumm_m[0] = pathensembles[0].tauavg['LML']-2
        taump_m[0] = pathensembles[0].tauavg['LMR']-2
        taupm_m[0] = pathensembles[0].tauavg['RML']-2
        taupp_m[0] = pathensembles[0].tauavg['RMR']-2
    else:
        taupp_m[0] = pathensembles[0].tauavg['RMR']-2

    # for [0+-] (i=1) LML
    taumm_m[1] = data[1]["pathlengths"]["LML"]["mean"] - 2
    # for [0+-] (i=1) LMR -> just 0
    # for [0+-] (i=1) RML -> just 0

    return taumm_m, taump_m, taupm_m, taupp_m


In [ ]:
def print_all_tau(pathensembles, taumm, taump, taupm, taupp):
    # print all tau
    print(f"                  mm            mp            pm            pp")
    for i in range(len(pathensembles)):
        print(f"{i} {pathensembles[i].name[-3:]}  {taumm[i]:13.1f} {taump[i]:13.1f} {taupm[i]:13.1f} {taupp[i]:13.1f}")

In [ ]:
taumm, taump, taupm, taupp = collect_tau(pathensembles, data)
taumm1, taump1, taupm1, taupp1 = collect_tau1(pathensembles, data)
taumm2, taump2, taupm2, taupp2 = collect_tau2(pathensembles, data)
taumm_m, taump_m, taupm_m, taupp_m = collect_taum(pathensembles, data)

In [ ]:
print("tau")
print_all_tau(pathensembles, taumm, taump, taupm, taupp)
print("\ntau1")
print_all_tau(pathensembles, taumm1, taump1, taupm1, taupp1)
print("\ntau_m")
print_all_tau(pathensembles, taumm_m, taump_m, taupm_m, taupp_m)
print("\ntau2")
print_all_tau(pathensembles, taumm2, taump2, taupm2, taupp2)

In [ ]:
def construct_tau_vector(N, NS, taumm, taump, taupm, taupp):
    assert N>=4
    assert NS==4*N-5
    assert len(taumm) == N
    assert len(taump) == N
    assert len(taupm) == N
    assert len(taupp) == N
    # unravel the values into one vector
    tau = np.zeros(NS)
    # [0-]
    tau[0] = taupp[0]
    # [0+-]
    tau[1] = taumm[1]
    tau[2] = taump[1]
    tau[3] = taupm[1]
    # [1+-] etc
    for i in range(1,N-2):
        tau[4*i]   = taumm[i+1]
        tau[4*i+1] = taump[i+1]
        tau[4*i+2] = taupm[i+1]
        tau[4*i+3] = taupp[i+1]
    # [(N-2)^(-1)]
    tau[-3] = taumm[-1]
    tau[-2] = taump[-1]
    # B
    tau[-1] = 0.   # whatever
    return tau

In [ ]:
tau  = construct_tau_vector(N, NS, taumm, taump, taupm, taupp)
tau1 = construct_tau_vector(N, NS, taumm1, taump1, taupm1, taupp1)
taum = construct_tau_vector(N, NS, taumm_m, taump_m, taupm_m, taupp_m)
tau2 = construct_tau_vector(N, NS, taumm2, taump2, taupm2, taupp2)
tau_m = tau-tau1-tau2  # yes, this is the same thing

print("tau")
print(tau)
print("\n")
print("tau1")
print(tau1)
print("taum")
print(taum)
print("tau2")
print(tau2)

print("\n")
print("tau = tau1+taum+tau2 => difference is", np.sum((tau-tau1-taum-tau2)**2))

# Compute tau for [0+]

In [ ]:
g1, g2, h1, h2 = vector_G(M, tau1, tau_m, tau2) #, doprint=True)
print("G")
print_vector(g1, labels1)
print_vector(g2, labels2)
print("H")
print_vector(h1, labels1)
print_vector(h2, labels2)
print("interesting")
print(h1[0])

In [ ]:
print(h1[0])

# Flux

In [ ]:
flux = 1/(tau[0]+h1[0])
# dt = 0.0002
# dt = 0.00002
dt = 0.01
flux
print(flux/dt, "1/time")

## MSM insights

In [ ]:
import deeptime.markov as dpt

msm = dpt.msm.MarkovStateModel(M)
print(msm)

tt = dpt.pcca(M, 2)